<a href="https://colab.research.google.com/github/Johnny-DF26/Machine_Learning/blob/master/Tunning_e_Validacao_Cruzada_(Algoritmos_de_Classifica%C3%A7%C3%A3o).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np

## Risco Credito

In [71]:
risco_credito = pd.read_csv('/content/credit_data_formatado.csv')
risco_credito

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


In [73]:
X_credito = risco_credito.drop(['default','clientid'], axis=1).values
y_credito = risco_credito.default.values
y_credito

array([0, 0, 0, ..., 1, 0, 0])

In [74]:
from sklearn.preprocessing import StandardScaler

padronizacao = StandardScaler()
X_credito_padronizado = padronizacao.fit_transform(X_credito)
X_credito_padronizado

array([[ 1.45393393,  1.36538005,  1.20281942],
       [-0.76217555,  0.54265932,  0.69642695],
       [ 0.83682073,  1.67417101,  1.17471147],
       ...,
       [-0.07122592, -0.97448606,  0.35420081],
       [-0.11000289,  1.73936652, -0.92675625],
       [ 1.682986  ,  1.14917551,  0.96381038]])

## Tuning dos Algoritmos e Validação Cruzada


In [245]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold

In [452]:
## Chamar a função com os dados pre-processados e padronizados para que os algoritmos possa fazer o treinamento 
def algoritmos(X, y):

    # KNN
    parametros_knn = {'n_neighbors':[3,5,7,9], 'p': [1,2]}
    grid_search_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=parametros_knn)
    grid_search_knn.fit(X, y)
    param_knn = grid_search_knn.best_params_
    
    # Naive Bayes
    # parametros_knn = {}
    # grid_search = GridSearchCV(estimator=GaussianNB(), param_grid=parametros_knn)
    # grid_search.fit(X, y)
    # param_naive_bayes = grid_search.best_params_}')
    # print(f'Score: {grid_search.best_score_ *100}%')

    # Regressão Logistica
    logistica = {'tol': [0.01, 0.001, 0.0001, 0.00001], 
                 'C' :[1.0, 1.5, 2.0]}
    grid_search_logistica = GridSearchCV(estimator=LogisticRegression(), param_grid=logistica)
    grid_search_logistica.fit(X, y)
    regressao_logistica = grid_search_logistica.best_params_

    # Arvore de Decisão
    arvore = {'criterion':['entropy', 'gini'], 'splitter':['best', 'random'], 
              'min_samples_split':[2,5,10], 
              'min_samples_leaf':[1,2,5,10]}
    grid_search_arvore = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=arvore)
    grid_search_arvore.fit(X, y)
    param_arvore = grid_search_arvore.best_params_
    

    # Arvore Randomica
    arvore_random = {'criterion':['entropy', 'gini'],'n_estimators':[20,40,60,100,150,200], 
              'min_samples_split':[2,5,10], 
              'min_samples_leaf':[1,2,5,10]}
    grid_search_arvore_random = GridSearchCV(estimator=RandomForestClassifier(), param_grid=arvore_random)
    grid_search_arvore_random.fit(X, y)
    param_arvore_random = grid_search_arvore_random.best_params_
    
    # SVC
    svc = {'C':[1.0, 1.5, 2.0], 'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
           'tol':[0.1, 0.01, 0.001, 0.0001, 0.00001]}
    grid_search_svc = GridSearchCV(estimator=SVC(), param_grid=svc)
    grid_search_svc.fit(X, y)
    param_svc = grid_search_svc.best_params_

    # Redes Neurais
    rna = {'solver' : ['lbfgs', 'sgd', 'adam'],'tol':[0.1, 0.01, 0.001, 0.0001, 0.00001], 
           'activation':['logistic', 'tanh', 'relu'], 'max_iter': [500, 1000]}
    grid_search_rna = GridSearchCV(estimator=MLPClassifier(), param_grid=rna)
    grid_search_rna.fit(X, y)
    param_rna = grid_search_rna.best_params_

    validacao_cruzada(param_knn,regressao_logistica, param_arvore, param_arvore_random, param_svc, param_rna, X, y)


def validacao_cruzada(param_knn,regressao_logistica, param_arvore, param_arvore_random, param_svc, param_rna, x, y):

    resultados_knn = []
    resultados_naive_bayes = []
    resultados_logistica = []
    resultados_arvore = []
    resultados_arvore_random = []
    resultados_svc = []
    resultados_rna = []

    for i in range(30):
        
        kfold = KFold(n_splits=30, random_state=i, shuffle=True)
        #KNN
        knn = KNeighborsClassifier(n_neighbors=list(param_knn.values())[0], p=list(param_knn.values())[1])
        score = cross_val_score(knn, x, y, cv=kfold)
        resultados_knn.append(score.mean())

        # Naive Bayes
        naive_bayes = GaussianNB()
        score = cross_val_score(naive_bayes, x, y, cv=kfold)
        resultados_naive_bayes.append(score.mean())

        # Regresssao Logistica
        logistica = LogisticRegression(C=list(regressao_logistica.values())[0], tol=list(regressao_logistica.values())[1])
        score = cross_val_score(logistica, x, y, cv=kfold)
        resultados_logistica.append(score.mean())

        # Arvore 
        arvore = DecisionTreeClassifier(criterion=list(param_arvore.values())[0], 
                                        splitter= list(param_arvore.values())[1], 
                                        min_samples_split= list(param_arvore.values())[2],
                                        min_samples_leaf=list(param_arvore.values())[3])
        score = cross_val_score(arvore, x, y, cv=kfold)
        resultados_arvore.append(score.mean())

        # Arvore Random
        arvore_random = RandomForestClassifier(criterion=list(param_arvore_random.values())[0],
                                               n_estimators=list(param_arvore_random.values())[1],
                                               min_samples_split= list(param_arvore_random.values())[2],
                                               min_samples_leaf=list(param_arvore_random.values())[3])
        score = cross_val_score(arvore_random, x, y, cv=kfold)
        resultados_arvore.append(score.mean())

        # Arvore SVC
        svc = SVC(C=list(param_svc.values())[0], kernel= list(param_svc.values())[1], tol= list(param_svc.values())[2])
        score = cross_val_score(svc, x, y, cv=kfold)
        resultados_svc.append(score.mean())

        # Rede Neural
        rna = MLPClassifier(solver= list(param_rna.values())[0], tol= list(param_rna.values())[1], 
                            activation= list(param_rna.values())[2], max_iter= list(param_rna.values())[3])
        score = cross_val_score(rna, x, y, cv=kfold)
        resultados_rna.append(score.mean())

    resultados = pd.DataFrame({'KNN': resultados_knn, 'Naive Bayes': resultados_naive_bayes, 'Regresssao Logistica': resultados_logistica,
                               'Arvore': resultados_arvore, 'Arvore Random': resultados_arvore_random, 
                               'SVC': resultados_svc, 'Rede Neural': resultados_rna})
    
    print(resultados)

In [ ]:
algoritmos(X_credito_padronizado, y_credito)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron